# Автодополнение текста: LSTM и Transformer

Этот ноутбук запускает полный пайплайн проекта:

1. Подготовка датасета (train / val / test) из `data/raw_dataset.txt`.
2. Обучение LSTM-модели автодополнения и замер метрики ROUGE.
3. Оценка трансформера `distilgpt2` из `Transformers` и замер метрики ROUGE + вывод примеров предсказаний.

Просто выполните следующую ячейку, чтобы последовательно запустить все шаги.


In [ ]:
from pathlib import Path

from data_utils import load_and_preprocess_text, train_val_test_split

DATA_DIR = Path("./data")
RAW_PATH = DATA_DIR / "raw_dataset.txt"

print("1/3: Подготовка датасета из raw_dataset.txt...")

if not RAW_PATH.exists():
    raise FileNotFoundError(
        f"Не найден файл с сырыми данными: {RAW_PATH}. Поместите raw_dataset.txt в папку data."
    )

splittedtext = load_and_preprocess_text(str(RAW_PATH))
train_tokens, val_tokens, test_tokens = train_val_test_split(splittedtext)

DATA_DIR.mkdir(parents=True, exist_ok=True)

with (DATA_DIR / "train_tokens.txt").open("w", encoding="utf-8") as f_train:
    for sent in train_tokens:
        f_train.write(" ".join(sent) + "\n")

with (DATA_DIR / "val_tokens.txt").open("w", encoding="utf-8") as f_val:
    for sent in val_tokens:
        f_val.write(" ".join(sent) + "\n")

with (DATA_DIR / "test_tokens.txt").open("w", encoding="utf-8") as f_test:
    for sent in test_tokens:
        f_test.write(" ".join(sent) + "\n")

print(
    f"Готово. Кол-во предложений: train={len(train_tokens)}, val={len(val_tokens)}, test={len(test_tokens)}"
)


In [ ]:
from lstm_train import train_lstm_model

print("2/3: Обучение LSTM-модели и замер ROUGE на валидации...")
train_lstm_model()
print("Обучение LSTM завершено, модель сохранена в models/lstm_autocomplete.pt")



In [ ]:
from eval_transformer_pipeline import evaluate_distilgpt2_on_val

print("3/3: Оценка трансформера distilgpt2 на валидационной выборке...")
evaluate_distilgpt2_on_val()
print("Оценка distilgpt2 завершена.")

